## **Predictive ML Project:3I/ATLAS(Interstellar Object)**

#### **Step 0 Import Libraries**

In [21]:
import pandas as pd
import numpy as py
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates
from statsmodels.tsa.arima.model import ARIMA
from astroquery.jplhorizons import Horizons

## **Project Introduction**

##### Predicting the Motion of 3I/ATLAS
-**Objective**: Predict future distance & velocity of interstellar object 3I/ATLAS
-**Dataset Source**: NASA JPL Horizons
-**Approach**: Time-series forecasting & regression modeling
-I will do:
1. Collect ephemeris data (2019-2025).
2. Perform EDA(distance/velocity trends).
3. Train ML models(ARIMA, Prophet, Random Forest).
4. Comapre predictions vs actual 

## **2.Data Collection**

In [4]:
obj = Horizons(id='A/2019 J2', location = '500@0',
         epochs={'start':'2019-01-01',
                 'stop':'2025-01-01',
                 'step':'30d'})

eph = obj.ephemerides()

df = eph.to_pandas()
df.head()

,targetname,datetime_str,datetime_jd,M1,M2,k1,k2,phasecoeff,solar_presence,interfering_body,...,r_3sigma,r_rate_3sigma,SBand_3sigma,XBand_3sigma,DoppDelay_3sigma,true_anom,hour_angle,alpha_true,PABLon,PABLat
0,Palomar (C/2019 J2),2019-Jan-01 00:00,2458484.5,9.4,17.4,22.5,5.0,0.03,,,...,125280.8630,0.010138,145.02,527.07,0.835784,278.6682,<NA>,0.1370,20.8750,16.7316
1,Palomar (C/2019 J2),2019-Jan-31 00:00,2458514.5,9.4,17.4,22.5,5.0,0.03,,,...,102047.8280,0.009234,139.38,506.56,0.680790,285.3827,<NA>,0.1546,18.8998,23.1812
2,Palomar (C/2019 J2),2019-Mar-02 00:00,2458544.5,9.4,17.4,22.5,5.0,0.03,,,...,79894.8818,0.008115,131.52,478.01,0.533001,293.5502,<NA>,0.1748,16.2192,30.9899
3,Palomar (C/2019 J2),2019-Apr-01 00:00,2458574.5,9.4,17.4,22.5,5.0,0.03,,,...,59347.6888,0.006717,119.21,433.27,0.395925,303.5807,<NA>,0.1971,12.2465,40.4887
4,Palomar (C/2019 J2),2019-May-01 00:00,2458604.5,9.4,17.4,22.5,5.0,0.03,,,...,41436.9105,0.004977,98.50,357.99,0.276437,315.9085,<NA>,0.2197,5.4723,51.9038


## **3.Data Cleaning & Preparation**

In [25]:
possible_mag_cols=['V','M1','Tmag','Nmag']
mag_col = None
for col in possible_mag_cols:
    if col in df.columns:
        mag_col = col
        break
if mag_col is None:
    raise ValueError("No magnitude column found in dataframe!") 

data = df[['datetime_str','r','delta',mag_col]].copy()
data.rename(columns={'r':'heliocentric_dist',
            'delta':'geocentric_dist',
               mag_col : 'magnitude'},inplace=True)

data['datetime_str'] = pd.to_datetime(data['datetime_str'])

data.set_index('datetime_str',inplace=True)

print(data.head())           

              heliocentric_dist  geocentric_dist  magnitude
datetime_str                                               
2019-01-01             2.999213         3.001055        9.4
2019-01-31             2.728128         2.729472        9.4
2019-03-02             2.466763         2.467524        9.4
2019-04-01             2.223208         2.223301        9.4
2019-05-01             2.009836         2.009193        9.4


## **4.Exploratory Data Analysis (EDA)**

In [32]:
vectors = obj.vectors().to_pandas()
print(vectors.columns)


Index(['targetname', 'datetime_jd', 'datetime_str', 'M1', 'M2', 'k1', 'k2',
       'phasecoeff', 'x', 'y', 'z', 'vx', 'vy', 'vz', 'lighttime', 'range',
       'range_rate'],
      dtype='object')


In [29]:
# Check available columns
print(data.columns)

# Plot radial velocity relative to Earth
if 'delta_rate' in data.columns:
    plt.figure(figsize=(12,6))
    plt.plot(data.index, data['delta_rate'], color='orange')
    plt.xlabel("Date")
    plt.ylabel("Radial Velocity relative to Earth (km/s)")
    plt.title("3I/ATLAS Radial Velocity Over Time")
    plt.show()
else:
    print("Column 'delta_rate' not found in ephemeris.")


Index(['heliocentric_dist', 'geocentric_dist', 'magnitude'], dtype='object')
Column 'delta_rate' not found in ephemeris.


In [36]:
# Fetch vector data
obj = Horizons(id='A/2019 J2', location='500@0', epochs={'start':'2019-09-01', 
                                                         'stop':'2025-01-01', 
                                                         'step':'10d'})
vectors = obj.vectors().to_pandas()

# Compute velocity magnitude (km/s)
vectors['vel_mag'] = np.sqrt(vectors['VX']**2 + vectors['VY']**2 + vectors['VZ']**2)

# Convert datetime
vectors['datetime'] = pd.to_datetime(vectors['datetime_jd'], unit='D', origin='julian')

# Plot 3D velocity magnitude
plt.figure(figsize=(12,6))
plt.plot(vectors['datetime'], vectors['vel_mag'], color='red')
plt.xlabel("Date")
plt.ylabel("Velocity Magnitude (km/s)")
plt.title("3I/ATLAS 3D Space Velocity Over Time")
plt.show()


KeyError: 'VX'